### 0. Import the packages

In [69]:
import os,sys,shutil,mmap
import pickle as pickle
import numpy as np                                       # fast vectors and matrices
import matplotlib.pyplot as plt                          # plotting
from scipy.fftpack import fft

import time

sys.path.insert(0,'lib/')
import config

from sklearn.metrics import average_precision_score

import tensorflow as tf

%matplotlib inline

notebook_reference = str(190121)

In [70]:
debugging = False
delete_ids = [2560, 1792, 2562, 2506, 2564, 2566, 1793, 2568, 2570, 2571, 2572, 2573, 2575, 2576, 2302, 2581, 2582, 
              2586, 2075, 2076, 2077, 2590, 2079, 2080, 2081, 2594, 2595, 2596, 2603, 2678, 2397, 2608, 2611, 2614, 
              2104, 2105, 2619, 2620, 2621, 2622, 2112, 2113, 2114, 2627, 2116, 2629, 2118, 2119, 2632, 2633, 2127, 
              2131, 2138, 2319, 2140, 2405, 2659, 2148, 2149, 2150, 2151, 2154, 2155, 2156, 2157, 2158, 2159, 2160, 
              2161, 2677, 2166, 2167, 2168, 2169, 2239, 2325, 2177, 2178, 2179, 2180, 1729, 2186, 2242, 2194, 2195, 
              2196, 1829, 2198, 2200, 2201, 2202, 2203, 2204, 2330, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 
              2215, 2588, 2218, 2219, 2220, 2221, 2222, 2224, 2225, 2227, 2228, 2229, 2230, 2231, 2232, 2234, 2591, 
              2237, 2238, 1727, 1728, 2241, 1730, 2243, 2244, 1733, 1734, 2593, 2248, 1739, 2240, 2082, 1742, 2083, 
              1749, 1750, 1751, 1752, 1755, 1756, 1757, 1758, 1760, 1763, 1764, 1765, 1766, 1768, 2282, 1771, 1772, 
              1773, 1775, 1776, 1777, 2292, 2293, 2294, 2295, 2296, 2297, 1919, 1788, 1789, 1790, 2304, 2305, 2307, 
              2308, 2310, 2313, 2314, 2315, 1805, 2318, 1807, 2320, 2322, 1811, 1812, 1813, 1817, 1818, 2334, 2335, 
              2336, 1828, 2341, 2342, 2343, 2567, 2345, 2346, 1835, 2348, 2350, 2491, 2357, 2358, 2359, 2364, 2365, 
              2366, 2368, 2371, 2372, 2373, 2374, 2529, 2376, 2377, 2379, 2443, 2381, 2383, 2384, 1873, 2388, 2389, 
              2390, 2391, 2392, 2393, 2618, 2398, 2403, 2404, 1893, 2406, 2410, 2411, 2415, 2417, 2147, 2451, 2420, 
              2422, 2423, 2424, 1916, 2538, 1918, 2431, 2432, 2433, 1922, 1923, 2436, 2441, 2442, 1931, 1932, 2626, 
              1735, 2285, 1859, 2247, 2117, 2288, 2466, 2444, 2289, 2472, 2473, 2476, 2477, 2478, 2480, 2481, 2482, 
              2483, 2462, 2486, 2487, 2488, 2490, 2463, 2492, 2494, 2555, 2497, 1822, 2501, 2502, 2504, 2505, 2078, 2507]

### 1. Import the data
Import the recording and labels of the 331 recordings from the data base.

#### 1.1 Import the labels of the data set.
The 331 labels are saved in a dictionary whose
- keys are integers (the ids of the recordings)
- values are intervaltrees (for the different intervals of the recording it gives the played notes between 1 and 128)

In [71]:
with open(config.labels_path, 'rb') as f:
    labels = pickle.load(f)

In [72]:
print(len(labels), 'labels')

331 labels


In [73]:
for key, value in labels.items() :
    if key == 2239:
        # id of the recording
        print (key)
        # intervaltree containing for each note intervals in which it is played
        print (type(value))

2239
<class 'intervaltree.intervaltree.IntervalTree'>


#### 1.2 Import recordings

Create a dictionary containing the data (recordings). Use Memory-mapped file objects.
For each recording data[rec_id] is a tuple:
1. data[rec_id][0] is the memory-mapped file object to the recording. It can be accesed by 
~~~~~~
np.frombuffer(data[rec_id][0], dtype=np.float32).copy()
~~~~~~
2. data[rec_id][1] is its length (float)

In [74]:
sz_float = 4

In [75]:
data = dict()
for record in os.listdir(config.records_path):
    fd = os.open(config.records_path + record, os.O_RDONLY)
    buff = mmap.mmap(fd, 0, mmap.MAP_SHARED, mmap.PROT_READ)
    data[int(record[:-4])] = (buff, len(buff)/sz_float)
    os.close(fd)

In [76]:
for key, value in data.items() :
    if key == 2494:
        # id of the recording
        print ('Key:',key,', Format:',type(key))
        # its value is a tuple
        print ('Value:',value)
        # 1. value: reference to recording
        print ("1. Entry: Memory-mapped file objects ,", data[2239][0])
        # to access content use:
        x = np.frombuffer(data[2239][0], dtype=np.float32).copy()
        print('For example:')
        print(x[3005:3017])
        print ("2. Entry: length of recording",str(data[2239][1]),", Type:",type(data[2239][1]))
        #print(len(x))

Key: 2494 , Format: <class 'int'>
Value: (<mmap.mmap object at 0x7f99a96f1b70>, 13100544.0)
1. Entry: Memory-mapped file objects , <mmap.mmap object at 0x7f9fd4d3c9c0>
For example:
[0.00036621 0.00039673 0.00036621 0.00036621 0.00033569 0.00033569
 0.00036621 0.00036621 0.00036621 0.00033569 0.00036621 0.00039673]
2. Entry: length of recording 5293440.0 , Type: <class 'float'>


#### 1.3 Split up in the test and train set

In [77]:
extended_test_set = True
use_mirex = True
    
# use the small or big test set
if extended_test_set:
    test_ids = config.test_ids_ext
else:
    test_ids = config.test_ids

# include the mirex dev set in test results
if use_mirex:
    test_ids = test_ids + config.mirex_id

train_ids = [rec_id for rec_id in labels.keys() if rec_id not in test_ids]

if debugging == True:
    train_ids = [rec_id for rec_id in train_ids if rec_id not in delete_ids]


print('There are {} samples in the test set'.format(len(test_ids)))
print('There are {} samples in the train set'.format(len(train_ids)))

There are 11 samples in the test set
There are 320 samples in the train set


### 2. Preprocessing

The input data (recordings) are normalized and the possible note range is defines to 128 notes. The lowest note is ..., the highest ... .

In [78]:
# Normalize the recordings
normalize = True
# Possible notes are from range [base_note, base_note + m)
base_note = 0
mm = 128

#### 2.1 Data augmentation

Always make two small changes to input data by randomly stretching or shrinking our input audio that are not noticable by human hearing, but augment the data set:
1. Pitch-shift in the frequency domain: Randomly  shifting  each data points in a minibatch by an integral number of semitones. This reinforces the architectural structure of the translation-invariant network.
2. Jittering: continuous shift to each data point. This makes the models more robust to tuning variation between recordings --> regularization

Define a function for extracting a sequence and its label as well as applying the data augmentation.

In [80]:
# function to obtain one data sample from recordings
# input:
# - rec_id: the id of the recording
# - timesteps: uneven integer. int(timesteps/2) gives the number of windows in each direction that are considered
# - s: frame-number to extract
# - window: windowsize
# - pitchshift: shift the label by pitchshift halftones
# - scaling_factor: scaling factor in frequency domain (includes corrections because of pitch shift and jitter)
# - normalize: boolean
# output
# - x: adjacent audio segments of shape (timesteps, window)
# - y: corresponsing labels, 0-1-vector of shape (timesteps, mm)
def get_data(rec_id,timesteps,s,window,pitch_shift,scaling_factor, normalize=True):
   
    x = np.frombuffer(data[rec_id][0][s*sz_float:int(
        s+scaling_factor*window)*sz_float*timesteps], dtype=np.float32).copy()
    
    if normalize: 
        x /= np.linalg.norm(x) + config.epsilon

    xp = np.arange(window*timesteps,dtype=np.float32)
    # interpolate the discrete function (np.arange(len(x)), x) at the points (scaling_factor*xp)
    x = np.interp(scaling_factor*xp,np.arange(len(x),dtype=np.float32),x).astype(np.float32)
    # real-valued audio frame with values in the range [−1,1], sampled at 44.1 kHz
    # -> x has length window
    
    y = np.zeros((timesteps,mm))
    # select center of frame
    for tt in range(timesteps):
        for label in labels[rec_id][s + (tt+1/2)*scaling_factor*window]:
            y[tt, label.data[1]+pitch_shift-base_note] = 1 # apply pitch shift here (in frequency domain)
    
    return np.reshape(x,(timesteps,window)),y

#### 2.3 Creation of Training and Test set for protocolling

Routines that create sample for the protcoling:
1. Routine to obtain the training samples from numpy-file.
2. Routine to obtain test samples given the recording IDs of the test samples.

In [82]:

# function to obtain training sample which is already saved in 
# numpy fromat in file. only normalization is necessary.
# input:
# - rec_ids: list of ids of training samples
# - timesteps: number of timesteps for each sequence
# - d: window size (integer)
# - normalize: boolean
# output:
# - Xtrain: training sample of size (nn, timesteps, 1, d, 1)
# - Ytrain: corresponding labels of size (nn, 128)
def get_training_sample(rec_ids, timesteps, d=16384, normalize=True):
    cachex = config.tmp + 'Xtrain_ext_{}_{}.npy'.format(d,timesteps)
    cachey = config.tmp + 'Ytrain_ext_{}_{}.npy'.format(d,timesteps)
    #shape Xtrain: (15000, 1, 16384, 1)
    #shape Ytrain: (15000, 128)
    if os.path.exists(cachex) and os.path.exists(cachey):
        Xtrain = np.load(cachex)
        Ytrain = np.load(cachey)
        
    else:
        print(cachex)
        print(cachey)
        
        start_time = time.time()
        
        nn_seq = []
        # skip in beginning of recording
        offset = 44100
        
        for i in range(len(rec_ids)):
            nn_seq += [min(int((data[rec_ids[i]][1] - offset - (timesteps*window))/window),100)]
        
        print(sum(nn_seq), "training samples of", timesteps," timesteps to calculate training error")
        
        
        Xtrain = np.zeros([sum(nn_seq),timesteps,1,window,1])
        Ytrain = np.zeros([sum(nn_seq),mm])
        
        index = 0

        for i in range(len(rec_ids)):
            count = nn_seq[i]
            print("recording",i,"has length",count)
            X = np.zeros([count,timesteps,1,window,1])
            Y = np.zeros([count,mm])
            
            # calculate the stride in case it is not fixed
            stride = int((data[rec_ids[i]][1] - offset - (timesteps*window))/count) 
            print("stride:",stride)
            
            for j in range(count):
                X[j,:,0,:,0], Y_tmp = get_data(rec_ids[i],timesteps,int(offset+j*stride),window,
                                           pitch_shift=0,scaling_factor=1)
                Y[j] = Y_tmp[int(timesteps/2)]

            Xtrain[index:index+count] = X
            Ytrain[index:index+count] = Y
            
              
        print('generated in {} seconds'.format(time.time()-start_time))
            
        if debugging == False:
            print("save xtrain to", cachex)
            print("size:",np.shape(Xtrain))
            np.save(cachex,Xtrain)
            print("save ytrain to", cachey)
            print("size:",np.shape(Ytrain))
            np.save(cachey,Ytrain)
        

    if normalize:
        Xtrain = Xtrain.reshape(len(Xtrain),timesteps,1,d,1)
        for i in range(len(Xtrain)):
            Xtrain[i] /= np.linalg.norm(Xtrain[i]) + config.epsilon

    return Xtrain,Ytrain


# function to obtain test sample using get_data(). every sample is split 
# up in count sequences.
# input:
# - rec_ids: list of ids of test samples
# - window: size of the input frame
# - count: number of sequences to be created for each sample
# - timesteps: number of sequences to be included in each sample (for rnn)
# - fixed_stride: -1 (stride is adapted to sample) or nonnegative integer
# - pitch_shift: value by which recording is stretched or shrinked
# output:
# - Xtest: test sample of size (count*len(rec_ids),timesteps,1,window,1)
# - Ytest: corresponding labels of size (count*len(rec_ids),mm) (only middle of sequence)
def get_test_sample(rec_ids, window, count, timesteps, fixed_stride=-1, pitch_shift=0):
    
    start_time=time.time()
    
    cachex = config.tmp + 'Xtest_ext_{}_{}.npy'.format(window,timesteps)
    cachey = config.tmp + 'Ytest_ext_{}_{}.npy'.format(window,timesteps)
    
    #print(cachex, cachey)
    
    if os.path.exists(cachex) and os.path.exists(cachey):
        #print('loaded from files in ')
        Xtest = np.load(cachex)
        Ytest = np.load(cachey)  
        #print('{} seconds'.format(time.time()-start_time))
         
    else:

        # get number of seqeunces
        nn_seq = len(rec_ids) * count

        Xtest = np.zeros([nn_seq,timesteps,1,window,1])
        Ytest = np.zeros([nn_seq,mm])

        # skip in beginning of recording
        offset = 44100

        # calculate scaling factor
        sf = 2.**(pitch_shift/12.)
        # for every sample create 1000 sequences
        for i in range(len(rec_ids)):
            X = np.zeros([count,timesteps,1,window,1])
            Y = np.zeros([count,mm])
            # calculate the stride in case it is not fixed
            stride = (data[rec_ids[i]][1] - offset - (timesteps*sf*window))/count if fixed_stride==-1 else fixed_stride
            for j in range(count):
                X[j,:,0,:,0], Y_tmp = get_data(rec_ids[i],timesteps,int(offset+j*stride),window,
                                           pitch_shift=int(round(pitch_shift)),scaling_factor=sf)
                Y[j] = Y_tmp[int(timesteps/2)]

            Xtest[i*count:(i+1)*count] = X
            Ytest[i*count:(i+1)*count] = Y
              
        print('generated in {} seconds'.format(time.time()-start_time))
            
        print("save xtest to", cachex)
        print("size:",np.shape(Xtest))
        np.save(cachex,Xtest)
        print("save ytest to", cachey)
        print("size:",np.shape(Ytest))
        np.save(cachey,Ytest)
        
    return Xtest, Ytest

Save the results, losses, precicions and weights at each protocol point inside a class.

In [83]:
class model_results():
    def __init__(self, test_ids, train_ids):
        # weights
        self.weights = dict()
        # step
        self.iter = 0
        # result statistics
        self.stats = dict()
        self.stats['iter'] = [False,'{:<8}',[]]
        # time that has passed since last record
        self.stats['time'] = [True,'{:<8.0f}',[]]
        self.stats['lr'] = [False,'{:<8.6f}',[]]
        # mean squared errors
        self.stats['mse_train_1'] = [True,'{:<16.6f}',[]]
        self.stats['mse_train'] = [True,'{:<16.6f}',[]]
        self.stats['mse_test'] = [True,'{:<16.6f}',[]]
        # precicions
        self.stats['avp_train'] = [True,'{:<16.6f}',[]]
        self.stats['avp_test'] = [True,'{:<16.6f}',[]]
        
        self.averages = []
        
        self.test_ids = test_ids
        self.train_ids = train_ids

    # Register the weights inside the list w_list. 
    # The list name_list contains the corresponding names of the wariables in w_list
    def register_weights(self, w_list, name_list):
        if len(w_list) != len(name_list):
            print("error: input lists need to have same length")
            
        for ii in range(len(w_list)):
            w = w_list[ii]
            name = name_list[ii]
            # register the norm of the weights
            self.stats['n'+name] = [False,'{:<8.3f}',[]] 
            # register the weights
            self.weights[name] = w
            
        return None
        
model_stats = model_results(test_ids, train_ids)


#### Variables:

In [84]:
# variables for input layer

# input size of segment, predict notes played at midpoint of segment
window=16384
# number of timesteps to be considered for memory cell
timesteps = 5
# size of regions that outputs the convolution from layer one (receptive field)
dd = 4096
#stride for first layer convolution
stride=512
# --> 25 regions

# variables for filterbank (= first layer)
# number of nodes in first layer (by filterbank get from dd nodes to kk nodes)
kk = 512 

#### Creation of Test set for protocolling

In [85]:

X,Y = get_test_sample(model_stats.test_ids, window, 1000, timesteps)

Y = Y[:,base_note:base_note+mm] # in general without effect
# shape Xtest: (1000*len(test_ids), timesteps, 1, 16384, 1)   -> (11000, 5, 1, 16384, 1)
# shape Ytest: (1000*len(test_ids), 128)
# extended test-set: 11 samples

print("test sample:")
print("1000 samples each recording")
print(len(model_stats.test_ids), "test recordings")

print("Xtest")
print(np.shape(X))
print("Ytest")
print(np.shape(Y))

del X,Y


           

test sample:
1000 samples each recording
11 test recordings
Xtest
(11000, 5, 1, 16384, 1)
Ytest
(11000, 128)


#### Creation of Training set for protocolling

In [86]:
# B) run model on training set

# 1. get training sample
X,Y = get_training_sample(model_stats.train_ids, timesteps, window)
Y = Y[:,base_note:base_note+mm] # in general without effect
# shape Xtrain: (15000, 1, 16384, 1)
# shape Ytrain: (15000, 128)


print("train sample:")
print("1000 samples each recording")
print(len(model_stats.train_ids), "train recordings")

print("Xtrain")
print(np.shape(X))
print("Ytrain")
print(np.shape(Y))


#del X,Y

data/Xtrain_ext_16384_5.npy
data/Ytrain_ext_16384_5.npy
32000 training samples of 5  timesteps to calculate training error
recording 0 has length 100
stride: 166079
recording 1 has length 100
stride: 207240
recording 2 has length 100
stride: 144536
recording 3 has length 100
stride: 81626
recording 4 has length 100
stride: 243021
recording 5 has length 100
stride: 177461
recording 6 has length 100
stride: 221398
recording 7 has length 100
stride: 209578
recording 8 has length 100
stride: 222331
recording 9 has length 100
stride: 236892
recording 10 has length 100
stride: 87397
recording 11 has length 100
stride: 148730
recording 12 has length 100
stride: 132751
recording 13 has length 100
stride: 122095
recording 14 has length 100
stride: 46513
recording 15 has length 100
stride: 151207
recording 16 has length 100
stride: 174465
recording 17 has length 100
stride: 188105
recording 18 has length 100
stride: 143834
recording 19 has length 100
stride: 107891
recording 20 has length 100
st

recording 188 has length 100
stride: 115702
recording 189 has length 100
stride: 201987
recording 190 has length 100
stride: 83538
recording 191 has length 100
stride: 145539
recording 192 has length 100
stride: 169477
recording 193 has length 100
stride: 194107
recording 194 has length 100
stride: 146875
recording 195 has length 100
stride: 103203
recording 196 has length 100
stride: 216467
recording 197 has length 100
stride: 120563
recording 198 has length 100
stride: 144744
recording 199 has length 100
stride: 156471
recording 200 has length 100
stride: 59565
recording 201 has length 100
stride: 133143
recording 202 has length 100
stride: 129710
recording 203 has length 100
stride: 176182
recording 204 has length 100
stride: 413793
recording 205 has length 100
stride: 190421
recording 206 has length 100
stride: 89263
recording 207 has length 100
stride: 99539
recording 208 has length 100
stride: 139249
recording 209 has length 100
stride: 94229
recording 210 has length 100
stride: 